In [1]:
%matplotlib inline

import numpy as np
from pandas import read_csv
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plot
import seaborn

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
import numpy

In [3]:
path = "data/uid_domains_full.csv"
data = read_csv(path, delimiter='\t')
data['domains'] = [','.join(json.loads(i)) for i in data['domains']]

In [5]:
data.head()

,uid,gender,age,domains
0,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,F,18-24,"news.yandex.ru,sotovik.ru,zebra-zoya.ru"
1,d502331d-621e-4721-ada2-5d30b2c3801f,M,25-34,"rsdn.ru,citieslist.ru,rutv.ru,interfax.ru,vand..."
2,d50237ea-747e-48a2-ba46-d08e71dddfdb,F,25-34,"ru.oriflame.com,adme.ru,cdn.etgdta.com,povar.r..."
3,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,F,25-34,"1obl.ru,nadietah.ru,translate-tattoo.ru"
4,d503c3b2-a0c2-4f47-bb27-065058c73008,M,>=55,"fedpress.ru,eleks.ru,prommetizcomplect.ru,ramb..."


In [4]:
text_all = data['domains'].values.tolist()

In [38]:
count_vec = CountVectorizer(tokenizer=lambda x: [i for i in x.split(',')], 
                            max_features=4000) 

In [39]:
matrix_count = count_vec.fit_transform(text_all)
matrix = matrix_count.toarray()

In [40]:
words = [x[0] for x in sorted(count_vec.vocabulary_.items(), key=lambda x: x[1])]

In [41]:
matrix_df = pd.DataFrame(matrix, columns=words)

In [42]:
matrix_df['uid'] = data['uid']
matrix_df = pd.merge(matrix_df, data, on='uid').drop(['domains'], axis=1)
# matrix_df = pd.get_dummies(matrix_df, columns=['gender'])

In [43]:
matrix_df = matrix_df.iloc[:,2:]

In [44]:
test_data = matrix_df[(matrix_df['gender'] == '-') & (matrix_df['age'] == '-')]
train_data = matrix_df[~((matrix_df['gender'] == '-') & (matrix_df['age'] == '-'))]

In [45]:
y_age = pd.factorize(train_data.age)[0]
y_gender = pd.factorize(train_data.gender)[0]

In [46]:
train_data.index = train_data['uid']
test_data.index = test_data['uid']

In [47]:
train_data.drop(['age','gender','uid'],inplace=True, axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
test_data.drop(['age','gender','uid'],inplace=True,axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [49]:
train_data = train_data.astype(float)
test_data = test_data.astype(float)

In [50]:
def cross_valid(matrix_counts, target, test_size=0.05, random_state=0):
    return train_test_split(matrix_counts, target, test_size=test_size, random_state=random_state)

In [18]:
def roc_curve(title, Y_test, predicted, label='?', pos_label=0):
    fpr, tpr, th = metrics.roc_curve(Y_test, predicted, pos_label=pos_label)
    plot.plot(fpr, tpr, label=label);
    plot.xlabel('False Positive'); 
    plot.ylabel('True Positive'); 
    plot.title(title)
    plot.legend(bbox_to_anchor=(1, 1), loc=2); 
    plot.xlim([0.0, 1.0]); plot.ylim([0.0, 1]);
    print 'Logistic Regression (', label, ') AUC = ', metrics.auc(fpr, tpr)

In [51]:
X_train, X_test, Y_train, Y_test = cross_valid(train_data, y_gender, random_state=5)

In [52]:
X_train_a, X_test_a, Y_train_a, Y_test_a = cross_valid(train_data, y_age, random_state=5)

In [ ]:
%%time
#model1 = RandomForestClassifier(n_estimators=700, n_jobs=4)
#model1.fit(X_train, Y_train)
model1_1 = GradientBoostingClassifier(n_estimators=1000)
model1_1.fit(X_train, Y_train)
# model2 = MultinomialNB(alpha=1)
# model2.fit(X_train, Y_train)
#model3 = LogisticRegression(C=1.0, penalty='l2')
#model3.fit(X_train, Y_train)
#model4 = SVC()
#model4.fit(X_train, Y_train)

In [22]:
%%time
#model2 = RandomForestClassifier(n_estimators=700, n_jobs=4)
#model2.fit(X_train_a, Y_train_a)
model2_1 = GradientBoostingClassifier(n_estimators=700)
model2_1.fit(X_train_a, Y_train_a)

CPU times: user 3h 10s, sys: 4.21 s, total: 3h 15s
Wall time: 3h 9s


In [54]:
#predict1 = model1.predict_proba(X_test)
predict1_1 = model1_1.predict_proba(X_test)
# predict2 = model2.predict_proba(X_test)
# predict3 = model3.predict_proba(X_test)
# predict4 = model4.predict_proba(X_test)

In [55]:
#print metrics.roc_auc_score(Y_test,predict1[:,1])
print metrics.roc_auc_score(Y_test,predict1_1[:,1])

0.753601805882


In [26]:
import pickle
# save the classifier
with open('GB_700classifier.pkl', 'wb') as f1:
    pickle.dump(model1_1, f1) 

with open('GB_700classifier.pkl', 'wb') as f2:
    pickle.dump(model2_1, f2) 
    
    

Работа с тестовой выборкой. Формирование предсказание

In [27]:
m1_pred_gender = model1_1.predict_proba(test_data)

In [28]:
pred_for_test_gender = ['M' if i.argmax() == 1 else 'F' for i in m1_pred_gender]

In [29]:
m2_pred_age = model2_1.predict_proba(test_data)

In [30]:
pred_for_test_age = []

In [31]:
for i in m2_pred_age:
    if i.argmax()==0:
        pred_for_test_age.append('18-24')
    elif i.argmax()==1:
        pred_for_test_age.append('25-34')
    elif i.argmax()==2:
        pred_for_test_age.append('>=55')
    elif i.argmax()==3:
        pred_for_test_age.append('45-54')
    else:
        pred_for_test_age.append('35-44')

In [32]:
sum_pred = pd.DataFrame()

In [33]:
sum_pred['gender']=pred_for_test_gender


In [34]:
sum_pred['age']=pred_for_test_age

In [35]:
sum_pred.index = test_data.index

In [36]:
sum_pred = sum_pred.sort_index()

In [37]:
sum_pred.to_csv('project01_gender-age.csv',sep='\t')